In [2]:
import pandas as pd
df = pd.read_csv("https://waf.cs.illinois.edu/discovery/berkeley.csv")
df

,Year,Major,Sex,Admission
0,1973,C,F,Rejected
1,1973,B,M,Accepted
2,1973,Other,F,Accepted
3,1973,Other,M,Accepted
4,1973,Other,M,Rejected
...,...,...,...,...
12758,1973,Other,M,Accepted
12759,1973,D,M,Accepted
12760,1973,Other,F,Rejected
12761,1973,Other,M,Rejected


In [3]:
#Anzeigen welche Werte in der Spalte "Admission" enthalten sind
print("Unique Admission-Werte:", df["Admission"].unique())

#Korrigiertes Mapping von "Accepted"/"Rejected" auf 0/1
df["Admitted_Num"] = df["Admission"].map({"Accepted": 1, "Rejected": 0})

#Gruppierung nach Gender und Berechnung der Zulassungsquote
agg_gender = (
    df
    .groupby("Sex")["Admitted_Num"]
    .agg(
        Total_Applications="count",      # Anzahl Bewerbungen pro Geschlecht
        Total_Admissions="sum"           # Summe der 1er-Codierung = Anzahl Zulassungen
    )
    .assign(Admission_Rate=lambda x: x["Total_Admissions"] / x["Total_Applications"])
    .reset_index()
)

print("\nAggregierte Tabelle nach Gender:")
print(agg_gender)


Unique Admission-Werte: ['Rejected' 'Accepted']

Aggregierte Tabelle nach Gender:
  Sex  Total_Applications  Total_Admissions  Admission_Rate
0   F                4321              1494        0.345753
1   M                8442              3738        0.442786


In [5]:
import altair as alt

#Altair-Balkendiagramm erzeugen
chart = alt.Chart(agg_gender).mark_bar().encode(
    x=alt.X('Sex:N', title='Geschlecht'),
    y=alt.Y('Admission_Rate:Q', title='Zulassungsrate', axis=alt.Axis(format='.0%'))
).properties(
    width=200,
    height=300,
    title='Zulassungsrate nach Geschlecht'
)

chart


alt.Chart(...)

In [7]:
#Pivot-Tabelle: Anzahl Bewerbungen, Anzahl Zulassungen und Zulassungsrate pro Major × Gender
pivot_major_gender = (
    df
    .groupby(["Major", "Sex"])["Admitted_Num"]
    .agg(
        Total_Applications="count",
        Total_Admissions="sum"
    )
    .assign(Admission_Rate=lambda x: x["Total_Admissions"] / x["Total_Applications"])
    .reset_index()
)

#Sortiere die Ergebnisse nach Major und dann nach Gender
pivot_major_gender = pivot_major_gender.sort_values(["Major", "Sex"]).reset_index(drop=True)

print(pivot_major_gender)


    Major Sex  Total_Applications  Total_Admissions  Admission_Rate
0       A   F                 108                89        0.824074
1       A   M                1138               825        0.724956
2       B   F                  25                17        0.680000
3       B   M                 560               353        0.630357
4       C   F                 593               201        0.338954
5       C   M                 325               120        0.369231
6       D   F                 375               131        0.349333
7       D   M                 417               138        0.330935
8       E   F                 393                94        0.239186
9       E   M                 191                53        0.277487
10      F   F                 341                25        0.073314
11      F   M                 373                22        0.058981
12  Other   F                2486               937        0.376911
13  Other   M                5438              2

In [12]:
#Altair: Facet-Chart – je Major eine kleine Grafik,in der die beiden Geschlechter untereinander angezeigt werden.
chart_major_gender = alt.Chart(pivot_major_gender).mark_bar().encode(
    y=alt.Y('Sex:N', title=None, sort=['F', 'M']),
    x=alt.X('Admission_Rate:Q', title='Zulassungsrate', axis=alt.Axis(format='%')),
    color=alt.Color('Sex:N', legend=None)
).properties(
    width=600,
    height=50  # Höhe pro Major
).facet(
    row=alt.Facet('Major:N', title=None, header=alt.Header(labelAngle=0, labelAlign='left'))
).resolve_scale(
    x='shared'
).configure_header(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=16
)

chart_major_gender


alt.FacetChart(...)

In [14]:
#Berechne die Zulassungsrate pro Major (unabhängig vom Geschlecht)
major_rates = (
    df
    .groupby("Major")["Admitted_Num"]
    .mean()
    .reset_index(name="Major_Rate")
)

#Zähle Anzahl der Bewerbungen pro Gender × Major
apps_gender_major = (
    df
    .groupby(["Major", "Sex"])["Admitted_Num"]
    .count()
    .reset_index(name="Applications")
)

#Join: füge Major_Rate zu apps_gender_major hinzu
apps_gender_major = apps_gender_major.merge(major_rates, on="Major")

# Berechne für jedes Geschlecht den gewichteten Durchschnitt der Major-Rates, gewichtet nach Anzahl der Bewerbungen in diesem Major.
weighted = (
    apps_gender_major
    .groupby("Sex")
    .apply(lambda grp: pd.Series({
        # Weighted Average = Σ(Applications_in_Major × Major_Rate) / Σ(Applications_in_Major)
        "Weighted_Avg_Major_Rate": (grp["Applications"] * grp["Major_Rate"]).sum() / grp["Applications"].sum(),
        "Total_Applications": grp["Applications"].sum()
    }))
    .reset_index()
)

print("Gewichteter Durchschnitt der Major-Zulassungsraten pro Gender:")
print(weighted)

#Altair-Bar-Chart: Vergleich der gewichteten Durchschnitts-Major-Raten
chart_weighted = alt.Chart(weighted).mark_bar().encode(
    x=alt.X("Sex:N", title="Geschlecht"),
    y=alt.Y("Weighted_Avg_Major_Rate:Q", title="Ø Zulassungsrate der gewählten Majors", axis=alt.Axis(format='%')),
    color=alt.Color("Sex:N", legend=None)
).properties(
    width=200,
    height=250,
    title="Ø Zulassungsquote der Majors, in die sich Bewerber*innen einschreiben"
)

chart_weighted


Gewichteter Durchschnitt der Major-Zulassungsraten pro Gender:
  Sex  Weighted_Avg_Major_Rate  Total_Applications
0   F                 0.357272              4321.0
1   M                 0.436890              8442.0


/tmp/ipykernel_556/2024932430.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda grp: pd.Series({


alt.Chart(...)

In [15]:
#Altair-Balkendiagramm mit geschlechterspezifischen Farben
chart = alt.Chart(agg_gender).mark_bar().encode(
    x=alt.X('Sex:N', title='Geschlecht'),
    y=alt.Y('Admission_Rate:Q', title='Zulassungsrate', axis=alt.Axis(format='.0%')),
    color=alt.Color(
        'Sex:N',
        scale=alt.Scale(
            domain=['F', 'M'],
            range=['#4C78A8', '#F58518']
        ),
        legend=None
    )
).properties(
    width=200,
    height=300,
    title='Zulassungsrate nach Geschlecht'
)

chart


alt.Chart(...)

In [17]:
#Altair: Aggregierte Balkengrafik nach Sex mit Label
bar_gender = alt.Chart(agg_gender).mark_bar().encode(
    x=alt.X('Sex:N', title='Geschlecht'),
    y=alt.Y('Admission_Rate:Q', title='Zulassungsrate', axis=alt.Axis(format='%')),
    color=alt.Color('Sex:N', legend=None)
)

text_gender = alt.Chart(agg_gender).mark_text(
    dy=-5,                     # verschiebt den Text etwas nach oben innerhalb des Balkens
    color='black'
).encode(
    x=alt.X('Sex:N'),
    y=alt.Y('Admission_Rate:Q'),
    text=alt.Text('Admission_Rate:Q', format='.0%')
)

chart_gender = (bar_gender + text_gender).properties(
    width=300,
    height=250,
    title='Aggregierte Zulassungsrate nach Geschlecht'
)

chart_gender


#Pivot-Tabelle: Major × Sex mit Admission_Rate
pivot_major_gender = (
    df
    .groupby(["Major", "Sex"])["Admitted_Num"]
    .agg(
        Total_Applications="count",
        Total_Admissions="sum"
    )
    .assign(Admission_Rate=lambda x: x["Total_Admissions"] / x["Total_Applications"])
    .reset_index()
)

#Altair: Facet-Chart mit Labeln in den Balken
bar_major_gender = alt.Chart(pivot_major_gender).mark_bar().encode(
    y=alt.Y('Sex:N', title=None, sort=['F', 'M']),
    x=alt.X('Admission_Rate:Q', title='Zulassungsrate', axis=alt.Axis(format='%')),
    color=alt.Color('Sex:N', legend=None)
)

text_major_gender = alt.Chart(pivot_major_gender).mark_text(
    dx=5,                      # verschiebt den Text leicht nach rechts zum Balkenende
    baseline='middle'
).encode(
    y=alt.Y('Sex:N'),
    x=alt.X('Admission_Rate:Q'),
    text=alt.Text('Admission_Rate:Q', format='.0%')
)

chart_major_gender = (
    (bar_major_gender + text_major_gender)
    .properties(
        width=600,
        height=50  
    )
    .facet(
        row=alt.Facet('Major:N', title=None, header=alt.Header(labelAngle=0, labelAlign='left'))
    )
    .resolve_scale(x='shared')
    .configure_header(
        labelFontSize=12,
        titleFontSize=14
    )
    .configure_title(
        fontSize=16
    )
)

chart_major_gender



alt.FacetChart(...)

In [19]:
#Bar-Chart (Bewerbungen, Farbe = Zulassungsrate)
bar = alt.Chart(pivot_major_gender).mark_bar().encode(
    y=alt.Y('Major:N', title='Fach', sort='-x'),
    x=alt.X('Total_Applications:Q', title='Anzahl Bewerbungen'),
    color=alt.Color('Admission_Rate:Q',
                    scale=alt.Scale(scheme='blues'),
                    legend=alt.Legend(title='Zulassungsrate')),
    tooltip=[
        alt.Tooltip('Major:N', title='Fach'),
        alt.Tooltip('Sex:N', title='Geschlecht'),
        alt.Tooltip('Total_Applications:Q', title='Bewerbungen'),
        alt.Tooltip('Admission_Rate:Q', title='Zulassungsrate', format='.1%')
    ]
)

#Text-Label mit Zulassungsrate
label = alt.Chart(pivot_major_gender).mark_text(
    align='left',
    baseline='middle',
    dx=5
).encode(
    y=alt.Y('Major:N'),
    x=alt.X('Total_Applications:Q'),
    text=alt.Text('Admission_Rate:Q', format='.0%')
)

#Layern: bar + label
layered_chart = (bar + label).properties(
    width=250,
    height=300
)

#Facetten nach Geschlecht
faceted_chart = layered_chart.facet(
    column=alt.Column('Sex:N', title='Geschlecht')
).configure_title(
    fontSize=16
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_header(
    labelFontSize=14,
    titleFontSize=14
).properties(
    title='Bewerbungen & Zulassungsraten nach Fach und Geschlecht'
)

faceted_chart


alt.FacetChart(...)

In [22]:
#Altair-Bar-Chart gewichteter Durchschnitt
base = alt.Chart(weighted).encode(
    x=alt.X("Sex:N", title="Geschlecht"),
    y=alt.Y("Weighted_Avg_Major_Rate:Q",
            title="Ø Zulassungsrate der gewählten Majors",
            axis=alt.Axis(format='%')),
    color=alt.Color("Sex:N",
                    scale=alt.Scale(domain=["F", "M"],
                                    range=["#4C78A8", "#F58518"]),
                    legend=None)
)

bars = base.mark_bar().properties(
    width=200,
    height=250,
    title="Ø Zulassungsquote der Majors, in die sich Bewerber*innen einschreiben"
)

labels = base.mark_text(
    dy=-10,        # Text etwas oberhalb der Balkenmitte
    color="black",
    fontSize=12
).encode(
    text=alt.Text('Weighted_Avg_Major_Rate:Q', format='.0%')
)

chart_weighted = bars + labels
chart_weighted


alt.LayerChart(...)